In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.imports import *
np.set_printoptions(linewidth=100)

import pandas as pd
df = pd.read_csv('/kaggle/input/dsaa-6100-titanic-dicision_tree/train.csv')
tst_df = pd.read_csv('/kaggle/input/dsaa-6100-titanic-dicision_tree/test.csv')
modes = df.mode().iloc[0] #每列众数（出现频率最高的数）
#发现有多个众数
#import random
#modes = df.mode().iloc[0]  # 获取所有列的众数
# 处理多个众数的情况
#modes_list = [tuple(row) for row in modes.values]  # 将多个众数保存为元组的列表
#selected_modes = random.choice(modes_list)  # 随机选择一个众数组合
#df.fillna(selected_modes, inplace=True)  # 填充数据

df.isna().sum()#查看缺失数据 isna（）返回布尔类型，缺失是True。.sum()求和，True=1,Faulse=0,总共多少缺失

df

In [ ]:
#预处理，对缺失值填充
def proc_data(df): # Define a function to process the data
    df['Fare'] = df.Fare.fillna(0) # 为啥？下一步不就覆盖了嘛
    df.fillna(modes, inplace=True) # 用modes中众数填充
#新建一个LogFare，是Fare+1后再log，防止为0的情况，这样可以缩小数据范围，对于较大数据使得数据更具正态特征及降低对建模的影响
    df['LogFare'] = np.log1p(df['Fare']) 
    df['Embarked'] = pd.Categorical(df.Embarked) #转换为分类数据类型，下面进行编码
    df['Sex'] = pd.Categorical(df.Sex)#分类，二分类，将字符串变成0 1 便于后续建模
proc_data(df) 
proc_data(tst_df) 
df

In [ ]:
#浅浅分个类
cats=["Sex","Embarked"]#分类特征，上面pd.Categorical(df.Embarked)转化为分类数据类型
conts=['Age', 'SibSp', 'Parch', 'LogFare',"Pclass"]#连续特征
dep="Survived" #目标变量

In [ ]:
#df.Sex.head() #查看 DataFrame 或 Series 的前5行

In [ ]:
import seaborn as sns

fig,axs = plt.subplots(1,2, figsize=(11,5))#fig 是一个代表整个图表的对象 axs是子图对象
sns.barplot(data=df, y=dep, x="Sex", ax=axs[0]).set(title="Survival rate")#条形图axs[0]第一个子图
sns.countplot(data=df, x="Sex", ax=axs[1]).set(title="Histogram");#计数图

In [ ]:
from numpy import random
from sklearn.model_selection import train_test_split

random.seed(6100) #每次运行代码时都会生成相同的随机数
trn_df,val_df = train_test_split(df, test_size=0.25)#分割成 train 和val验证集
trn_df[cats] = trn_df[cats].apply(lambda x: x.cat.codes)# 分类变量（categorical variables）进行编码
val_df[cats] = val_df[cats].apply(lambda x: x.cat.codes)

In [ ]:
def xs_y(df):
    xs = df[cats+conts].copy()
    return xs,df[dep] if dep in df else None #条件语句

trn_xs,trn_y = xs_y(trn_df)
val_xs,val_y = xs_y(val_df)

In [ ]:
preds = val_xs.Sex==0 # True =1
from sklearn.metrics import mean_absolute_error
mean_absolute_error(val_y, preds)

In [ ]:
df_fare = trn_df[trn_df.LogFare>0]
fig,axs = plt.subplots(1,2, figsize=(11,5))
sns.boxenplot(data=df_fare, x=dep, y="LogFare", ax=axs[0]) #箱线图
sns.kdeplot(data=df_fare, x="LogFare", ax=axs[1]); #核密度估计图（kdeplot）

In [ ]:
preds = val_xs.LogFare>2.6 #只是简单二分，使得preds=1与存活映射
mean_absolute_error(val_y, preds)


In [ ]:
#子集得分，更全面地评估分割是否有利于减少目标变量值的不一致性。 数值越低，表示越纯，数值是inpurioty
def _side_score(side, y):
    tot = side.sum() #布尔类型，也就是属于子集side的标准差*属于子集的样本数量
    if tot <= 1:
        return 0 #不足够提供信息
    return y[side].std() * tot  #y中满足side的元素的标准差*tot个数 
    #标准差体现目标y中分散程度即不一致性
    #如果子集内部数据相近，标准差就小，分类就越纯 tot是为了考虑子集的大小对纯度的影响

In [ ]:
def score(col, y, split):
     lhs = col <= split #小于等于split threshold在左侧分类，left_hands是一个布尔类型，
        #他的sum（）是左侧的样本数量 ~lhs是右侧的，Y是目标样本，len(y)总样本长度
     return (_side_score(lhs, y) + _side_score(~lhs, y)) / len(y)

In [ ]:
score(trn_xs["Sex"], trn_y, 0.5)
score(trn_xs["LogFare"], trn_y, 2.6)

In [ ]:
def iscore(nm, split):# 特征名，分割点  这个交互只能在kaggle
    col = trn_xs[nm]
    return score(col, trn_y, split)#

from ipywidgets import interact
interact(nm=conts, split=15.5)(iscore);#交互，允许用户通过滑块选择特征和分割点

In [ ]:
interact(nm=cats, split=2)(iscore);

In [ ]:
#以年龄为例，automatically找出年龄的最佳二分的threshold
nm = "Age"
col = trn_xs[nm]
unq = col.unique()#对col中值提取唯一值
unq.sort()#按升序排列
unq

In [ ]:
#计算在给定特征列（col）上进行二元分割的最佳分割点
scores = np.array([score(col, trn_y, o) for o in unq if not np.isnan(o)])
#列表推导式 if o in unq is not nan 就计算在o处分割的分数
unq[scores.argmin()]#argmin() 函数找到了 scores 数组中最小值的索引
#scores.argmin（）只返回第一个最小值，对于多个最小值，可以用np.where(scores == scores.min())[0][0]

In [ ]:
#鉴于上面想法，定义一个函数
# Define a function that finds the best binary split for a given column.
def min_col(df, nm):
    # Extract the desired column and the dependent variable from the dataframe.
    col, y = df[nm], df[dep]

    # Get unique non-null values from the column. dropna（）就求掉了nan的值
    unq = col.dropna().unique()

    # Calculate the score for each unique value, skipping NaN values.
    scores = np.array([score(col, y, o) for o in unq if not np.isnan(o)])

    # Find the index of the best (minimum) score.
    idx = scores.argmin()

    # Return the unique value associated with the best score and the best score itself.
    return unq[idx], scores[idx]
    #返回最佳的cutoff点的数据以及最佳点作为cutoff的得分
# Test the function on the "Age" column of the training dataframe.
min_col(trn_df, "Age")

In [ ]:
cols = cats+conts#分类型和连续型 cols 是一个包含列名的列表
{o:min_col(trn_df, o) for o in cols}#使用字典推导来创建一个字典
#键是 cols 列表中的列名 o，值是通过 min_col(trn_df, o) 函数计算得到的threshold和该分割值下对应的得分

In [ ]:
#使用布尔列表来筛选DataFrame，进行时，会返回一个true对应的新的dataframe
#原本是每个特征都直接找最佳分割点，现在是对某个特征分割后，再找对应的其他特征分界点
#先分男 女 男女之下再各自分
cols.remove("Sex")
ismale = trn_df.Sex==1 #=1 男
males,females = trn_df[ismale],trn_df[~ismale]
{o:min_col(males, o) for o in cols}

In [ ]:
{o:min_col(females, o) for o in cols}

发现对于男，下一个最好的二分方法是age,对于女，下一个更好的分界是Pclass。所以，之前做的工作1、确定在这几个特征中，首先按哪个分（更纯净），其次各自找最佳的split方法

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
#创建决策树分类器的类、导出决策树的可视化函数
m = DecisionTreeClassifier(max_leaf_nodes=4).fit(trn_xs, trn_y);
#m=决策树分类器对象,最大叶子节点的数量为 4，
#将训练数据 trn_xs（特征）和 trn_y（目标变量，通常是标签）传递给 m 对象
import graphviz
#export_graphviz包含在graphviz之中
#（决策树模型，dataframe用于提供特征名，尺寸默认10，宽高比默认0.6，节点中数值的小数精度默认2，允许传递额外的关键字参数给 export_graphviz 函数）
def draw_tree(t, df, size=10, ratio=0.6, precision=2, **kwargs):
    #使用 export_graphviz 函数生成决策树的可视化描述字符串（DOT 格式）
    #（决策树模型，输出文件默认none，表示不写出；特征名；节点是否填色；节点是否显示圆角矩形；
    #允许使用特殊字符；是否旋转；数值的小数精度；允许传递额外的参数）
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True, rounded=True,
                      special_characters=True, rotate=False, precision=precision, **kwargs)
    return graphviz.Source(re.sub('Tree {', f'Tree {{ size={size}; ratio={ratio}', s))
    #graphviz.Source（）创建可视化的图形对象，接受DOT格式的文本字符串s（描述图形的结构和外观）作为参数
    #re.sub('tree{',f'tree{{size={size}')是后面一个替换前面一个替换
    #f'  '是Python的f-string，用于将 size 和 ratio 的值嵌入到字符串中

In [ ]:
draw_tree(m, trn_xs, size=10)
#samples是这个节点样本一共多少，而value是实际死亡、存活的数量

In [ ]:
#gini的定义
def gini(cond):
    """
    Compute the Gini impurity for a specified condition.

    Parameters:
    - cond: A boolean condition for filtering the dataframe.

    Returns:
    - The Gini impurity value for the filtered data.
    """

    # Use the given condition to filter the dataframe and select the dependent variable column
    act = df.loc[cond, dep]

    # Calculate the Gini impurity:
    # Gini impurity is calculated as 1 minus the sum of the squared probabilities of each class.
    # In a binary classification (e.g., 0 and 1), if 'act.mean()' gives the probability of class '1',
    # then '1 - act.mean()' gives the probability of class '0'.
    # The formula then becomes: 1 - probability(1)^2 - probability(0)^2
    return 1 - act.mean()**2 - (1-act).mean()**2

In [ ]:
m = DecisionTreeClassifier(min_samples_leaf=50)
m.fit(trn_xs, trn_y)
draw_tree(m, trn_xs, size=10)


In [ ]:
mean_absolute_error(val_y, m.predict(val_xs))



In [ ]:
tst_df[cats] = tst_df[cats].apply(lambda x: x.cat.codes) #分类数据编码为数字
tst_xs,_ = xs_y(tst_df)#占位符_表示不需要目标变量

def subm(preds, suff):  #preds是模型的预测结果，suff是文件名的后缀
    tst_df['Survived'] = preds  #将预测结果preds添加到测试数据集的新列Survived中
    sub_df = tst_df[['PassengerId','Survived']]  #从测试数据集中选择两列
    sub_df.to_csv(f'submission.csv', index=False)  #保存为CSV文件，文件名为submission.csv
    #index=False表示不保存行索引
subm(m.predict(tst_xs), 'tree') #'tree'是一个后缀，用于构建输出文件名。
#训练好的模型m对测试集进行测试

In [ ]:
# the average of lots of uncorrelated random errors is zero.
# train each of them on a different random subset of the data
#随机森林
def get_tree(prop=0.85):
    """
    Generate and fit a decision tree classifier using a subset of the data.

    Parameters:
    - prop (float, optional): Proportion of the data to use for training. Default is 0.85.

    Returns:
    - A trained DecisionTreeClassifier.
    """

    # Determine the number of data points in the training set
    n = len(trn_y)

    # Randomly select a subset of indices based on the specified proportion
    idxs = random.choice(n, int(n*prop))

    # Train a DecisionTreeClassifier on the selected subset and return it
    return DecisionTreeClassifier(min_samples_leaf=5).fit(trn_xs.iloc[idxs], trn_y.iloc[idxs])

In [ ]:
trees = [get_tree() for t in range(100)]

In [ ]:
all_probs = [t.predict(val_xs) for t in trees]
avg_probs = np.stack(all_probs).mean(0)

mean_absolute_error(val_y, avg_probs)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(99, min_samples_leaf=5)
rf.fit(trn_xs, trn_y);
mean_absolute_error(val_y, rf.predict(val_xs))

In [ ]:
subm(rf.predict(tst_xs), 'rf')

In [ ]:
pd.DataFrame(dict(cols=trn_xs.columns, imp=m.feature_importances_)).plot('cols', 'imp', 'barh');